In [1]:
import pandas as pd    
import geopandas as gpd
import datetime
from geopandas.tools import geocode
#import geoplot
from shapely.geometry import Point
import numpy as np
import scipy.stats as stats
import scipy
import shapely
from shapely import speedups
speedups.enabled
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})

**GeoJson files**

In [5]:
#Census tracts
census_url = 'https://opendata.arcgis.com/datasets/de58dc3e1efc49b782ab357e044ea20c_9.geojson'
census_bounds = gpd.read_file(census_url)

In [6]:
#Bike racks
racks_url = 'https://opendata.arcgis.com/datasets/f86c29ce743e47819e588c3d643ceb63_0.geojson'
racks_data = gpd.read_file(racks_url)

In [7]:
#Parking -- street signs indicate parking zone and date of installation
street_signs_url = 'https://gisdata.seattle.gov/server/rest/services/SDOT/SDOT_Signs/MapServer/0/query?where=1%3D1&outFields=OBJECTID,COMPKEY,COMPTYPE,CATEGORY,CATEGORYDESCR,ADDDTTM,MODDTTM,INSTDATE,SHAPE_LNG,SHAPE_LAT&outSR=4326&f=json'
street_signs = gpd.read_file(street_signs_url)

In [8]:
#Population
pop_url_2010 = 'https://gisrevprxy.seattle.gov/arcgis/rest/services/CENSUS_EXT/CENSUS_2010_BASICS/MapServer/15/query?where=1%3D1&outFields=SHAPE,GEOID10,NAME10,ACRES_TOTAL,Total_Population,OBJECTID&outSR=4326&f=json'
#pop_url_2000
seattle_pop = pd.read_csv('City_Annual_Stats.csv')

_Traffic Flows_

In [9]:
url_list = ['https://opendata.arcgis.com/datasets/7015d5d46a284f94ac05c2ea4358bcd7_0.geojson',
            'https://opendata.arcgis.com/datasets/5fc63b2a48474100b560a7d98b5097d7_1.geojson',
            'https://opendata.arcgis.com/datasets/27af9a2485c5442bb061fa7e881d7022_2.geojson',
            'https://opendata.arcgis.com/datasets/4f62515558174f53979b3be0335004d3_3.geojson',
            'https://opendata.arcgis.com/datasets/29f801d03c9b4b608bca6a8e497278c3_4.geojson',
            'https://opendata.arcgis.com/datasets/a0019dd0d6464747a88921f5e103d509_5.geojson',
            'https://opendata.arcgis.com/datasets/40bcfbc4054549ebba8b5777bbdd40ff_6.geojson',
            'https://opendata.arcgis.com/datasets/16cedd233d914118a275c6510115d466_7.geojson',
            'https://opendata.arcgis.com/datasets/902fd604ecf54adf8579894508cacc68_8.geojson',
            'https://opendata.arcgis.com/datasets/170b764c52f34c9497720c0463f3b58b_9.geojson',
            'https://opendata.arcgis.com/datasets/2c37babc94d64bbb938a9b520bc5538c_10.geojson',
            'https://opendata.arcgis.com/datasets/a35aa9249110472ba2c69cc574eff984_11.geojson']

In [10]:
def get_gdf(year):
    '''Enter the desired year to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007.
    '''
    num = year-7
    gdf_year = gpd.read_file(url_list[num])
    if year == 11:
        gdf_year = gdf_year.rename(columns={"YEAR_" : 'YEAR'})
    if year == 12:
        gdf_year = gdf_year.rename(columns={'STDY_YEAR' : 'YEAR'})
    if year == 15 or year == 16:
        gdf_year = gdf_year.rename(columns={"COUNTAAWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'FIRST_STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 15:
            year_list = ['2015']*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 16:
            year_list = ['2016']*len(gdf_year)
            gdf_year['YEAR'] = year_list
    elif year == 17 or year == 18:
        gdf_year = gdf_year.rename(columns={"AWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 17:
            year_list = ['2017']*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 18:
            year_list = ['2016']*len(gdf_year)
            gdf_year['YEAR'] = year_list
    #df_year_AAWDT = df_year['AAWDT'].values
    #df_year_geobase = df_year['GEOBASID'].values
    #df_year_dist = df_year['SHAPE_Length'].values
    gdf_year = gdf_year[[ 'YEAR', 'AAWDT','geometry']]
    return gdf_year #, df_year_AAWDT, df_year_geobase, df_year_dist

In [11]:
#Generate chronological stack of all traffic data

#I'm thinking for this we will do a for loop that iteratively stacks the aligned columns for every year

In [12]:
df_SND = gpd.read_file('Street_Network_Database_SND.geojson')

**Define census tract coordinate polygon areas:**

In [13]:
census_columns = ['NAME10', 'SHAPE_Area', 'geometry']
census_bounds_cleaned = census_bounds[census_columns]

In [14]:
census_bounds_cleaned.head()

,NAME10,SHAPE_Area,geometry
0,25,1.059462e+07,"POLYGON ((-122.29602 47.69023, -122.29608 47.6..."
1,26,1.339838e+07,"POLYGON ((-122.30817 47.69031, -122.30947 47.6..."
2,56,3.212601e+07,"POLYGON ((-122.39300 47.63956, -122.39421 47.6..."
3,68,7.729233e+06,"POLYGON ((-122.35070 47.63994, -122.35130 47.6..."
4,60,1.413816e+07,"POLYGON ((-122.34279 47.64320, -122.34280 47.6..."


**Once you're able to load in the relevant geoJson files:**

Send a message in the slack which one you wanna do. For right now I'm working on how to classify the locations from these files based on the census tract polygons. At this point we have to process:

1. Bike racks
3. Parking
4. Population

For each estimator we want year and geometric location. For example, if you run gdf_year(X) and head the file you will see the geometric coordinates. Refer to the code below for bike lanes specifically for a loose example of how we've approached this (or however you'd like, we are just shooting for consistent time series data of the estimators and their corresponding coordinates).

For cleaning based on year, we just have to make sure we are only using the 2007-2018 data so we will have to further filter the DFs to reflect that

## Bike lanes ##

In [15]:
lane_columns = ['SEGMENT_TY', 'SNDSEG_UPD', 'SHAPE_Leng', 'geometry']
bike = df_SND[lane_columns]

In [16]:
# Creates list of years in each row
lanes_years = []
for i in bike['SNDSEG_UPD'].values:
    lanes_years.append(int(i[0:4]))
    
lane_years = lanes_years

In [18]:
bike['YEAR'] = pd.Series(lane_years)

In [19]:
#Specify segment type to extract, for walkways this will be 5
bike = bike[bike['SEGMENT_TY']==6]

In [20]:
bike_cleaned = bike.drop(columns=['SNDSEG_UPD', 'SEGMENT_TY'])

In [78]:
lane_by_tract = gpd.sjoin(census_bounds_cleaned, bike_cleaned, op='intersects')
lanes_years = lane_by_tract.dissolve(by=['YEAR','NAME10'])
lanes_years.drop(columns=['geometry','SHAPE_Area','index_right'])
lanes_years.tail(n=16)

geometry    SHAPE_Area  \
YEAR NAME10                                                                    
2017 96      POLYGON ((-122.37568 47.58615, -122.37568 47.5...  1.972780e+07   
     99      MULTIPOLYGON (((-122.37226 47.58332, -122.3722...  7.024407e+07   
2018 101     POLYGON ((-122.27668 47.57075, -122.27687 47.5...  2.685513e+07   
     81      POLYGON ((-122.33115 47.60584, -122.33180 47.6...  8.988184e+06   
     86      POLYGON ((-122.31415 47.61292, -122.31278 47.6...  7.656689e+06   
     99      MULTIPOLYGON (((-122.37226 47.58332, -122.3722...  7.024407e+07   
2019 53.02   POLYGON ((-122.30653 47.66126, -122.30725 47.6...  2.514797e+07   
     58.02   POLYGON ((-122.37489 47.64847, -122.37614 47.6...  3.326349e+07   
     67      MULTIPOLYGON (((-122.33810 47.62690, -122.3381...  1.327363e+07   
     78      POLYGON ((-122.28153 47.62605, -122.28255 47.6...  2.239329e+07   
     80.02   POLYGON ((-122.34349 47.61394, -122.34435 47.6...  2.465482e+06   
2020 101     POLYGON ((-122.27668 47.57075, -122.27687 47.5...  2.685513e+07   
     106     POLYGON ((-122.39776 47.55750, -122.39856 47.5...  2.718619e+07   
     49      POLYGON ((-122.34723 47.66499, -122.34738 47.6...  1.178876e+07   
     63      POLYGON ((-122.28153 47.62605, -122.28152 47.6...  2.488186e+07   
     78      POLYGON ((-122.28153 47.62605, -122.28255 47.6...  2.239329e+07   

             index_right  SHAPE_Leng  
YEAR NAME10                           
2017 96            19809  130.463476  
     99            30139  148.163251  
2018 101           32285  269.759130  
     81            33054  348.181761  
     86            32402   74.541606  
     99            32190  103.611745  
2019 53.02         31033  107.116326  
     58.02         33632  342.715121  
     67            33489  151.478478  
     78            33788   73.788256  
     80.02         18231  161.343101  
2020 101           34106   37.016696  
     106           33598  226.960952  
     49            34110   38.993437  
     63            33368  220.676407  
     78            33368  220.676407

In [101]:
lengths = pd.Series(lanes_years['SHAPE_Leng'].values)

year_lane = []
tract_lane = []

for i in range(np.size(lengths)):
    year_lane.append(lanes_years.index.values[i][0])
for k in range(np.size(lengths)):
    tract_lane.append(lanes_years.index.values[k][1])

df_lanes = pd.DataFrame(np.stack((year_lane,tract_lane,lengths)).T)
df_lanes.head()

,0,1,2
0,2004,100.02,138.15138427468574
1,2004,101,138.31049553972244
2,2004,102,57.39931473030976
3,2004,103,156.09571324091084
4,2004,105,160.7547674761567


## Pedestrian Walkways -- join geometry

In [301]:
walk_columns = ['SEGMENT_TY', 'SNDSEG_UPD', 'SHAPE_Leng', 'geometry']
walk = df_SND[walk_columns]
walk.head()

,SEGMENT_TY,SNDSEG_UPD,SHAPE_Leng,geometry
0,1,2007-04-22,291.426335,"LINESTRING (-122.32286 47.52981, -122.32287 47..."
1,1,2007-04-22,431.467882,"LINESTRING (-122.32400 47.61848, -122.32401 47..."
2,1,2004-05-19,162.446146,"LINESTRING (-122.29191 47.76490, -122.29126 47..."
3,6,2004-05-19,79.522621,"LINESTRING (-122.30780 47.61410, -122.30748 47..."
4,1,2012-10-12,643.835648,"LINESTRING (-122.38213 47.67455, -122.38474 47..."


In [326]:
walks_years = []
for k in walk['SNDSEG_UPD'].values:
    walks_years.append(int(k[0:4]))

walk_years = walks_years

In [306]:
walk['YEAR'] = pd.Series(walk_years)

In [307]:
walks = walk[walk['SEGMENT_TY']==5]

In [308]:
walks_cleaned = walks.drop(columns=['SEGMENT_TY'])

In [327]:
walks_data = walks_cleaned.dissolve('YEAR',aggfunc=sum)
walks_data.head()

,geometry,SHAPE_Leng
YEAR,,
2004,"MULTILINESTRING ((-122.31491 47.69172, -122.31...",340.837104
2005,"LINESTRING (-122.32653 47.64449, -122.32653 47...",412.348264
2008,"LINESTRING (-122.38645 47.54191, -122.38646 47...",465.983637
2014,"LINESTRING (-122.38163 47.69059, -122.38162 47...",665.738612
2015,"MULTILINESTRING ((-122.34646 47.53375, -122.34...",1396.052513


In [4]:
walks_by_tract = gpd.sjoin(census_bounds_cleaned, census_bounds, op='intersects')

NameError: name 'census_bounds_cleaned' is not defined

## Bike racks -- join geometry and aggregate values

In [333]:
racks_df = gpd.read_file('Bike_Racks.geojson')
racks_df.head()

,OBJECTID,COMPKEY,COMPTYPE,SEGKEY,DISTANCE,WIDTH,UNITID,UNITTYPE,UNITDESC,BIKE_FACILITY,...,SIDE,OWNERSHIP,CURRENT_STATUS,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,OVERRIDEYN,OVERRIDECOMMENT,SHAPE_LNG,SHAPE_LAT,geometry
0,1,519234,16,12044,240.0,30.0,BKR-2855,BKR,OLIVE WAY 0060 BLOCK NW SIDE ( 225) 51 FT SW/O...,SGL,...,NW,SDOT,INSVC,DISTRICT7,,N,None,-122.335519,47.613547,POINT (-122.33552 47.61355)
1,2,518964,16,17977,512.0,-28.0,BKR-1790,BKR,NE BOAT ST 0110 BLOCK SW SIDE ( 512) 363 FT NW...,SGL,...,SW,SDOT,INSVC,DISTRICT4,,N,None,-122.316016,47.652276,POINT (-122.31602 47.65228)
2,3,519236,16,12045,216.0,29.0,BKR-2895,BKR,OLIVE WAY 0070 BLOCK NW SIDE ( 196) 80 FT SW/O...,SGL,...,NW,SDOT,INSVC,DISTRICT7,,N,None,-122.334488,47.613973,POINT (-122.33449 47.61397)
3,4,519257,16,9538,310.0,-25.0,BKR-1107,BKR,CALIFORNIA AVE SW 0220 BLOCK W SIDE ( 280) 27 ...,SGL,...,W,SDOT,INSVC,DISTRICT1,,N,None,-122.386571,47.583156,POINT (-122.38657 47.58316)
4,5,640211,16,9479,509.0,-26.5,BKR-3674,BKR,BROOKLYN AVE NE 0410 BLOCK E SIDE ( 501) 10 FT...,ONST,...,E,SDOT,INSVC,DISTRICT4,,N,None,-122.314258,47.658240,POINT (-122.31426 47.65824)


## Parking ## -- need to filter out rows based on irrelevant street sign categories

In [246]:
park_times = []
for timestamp in street_signs['ADDDTTM'].values:
    x = timestamp[:10]
    x_time = datetime.datetime.fromtimestamp(int(x)).isoformat()
    x_year = (int(x_time[0:4]))
    park_times.append(x_year)

In [273]:
street_signs['PARK YEAR'] = pd.Series(park_times)

In [274]:
keep_list = ['PPP','PR','PRZ','PTIML','PINST','PCARPL']

## Population processing

In [55]:
pop_url_2010 = 'https://gisrevprxy.seattle.gov/arcgis/rest/services/CENSUS_EXT/CENSUS_2010_BASICS/MapServer/15/query?where=1%3D1&outFields=SHAPE,GEOID10,NAME10,ACRES_TOTAL,Total_Population,OBJECTID&outSR=4326&f=json'
#pop_url_2000 =

In [57]:
pop_2010 = gpd.read_file(pop_url_2010)

In [58]:
pop_2010.head()

,GEOID10,NAME10,ACRES_TOTAL,Total_Population,OBJECTID,geometry
0,53033002500,25,243.219083,2802.0,1,"POLYGON ((-122.29602 47.69023, -122.29604 47.6..."
1,53033002600,26,307.584420,4660.0,2,"POLYGON ((-122.30817 47.69031, -122.30793 47.6..."
2,53033005600,56,737.511727,6382.0,3,"POLYGON ((-122.39300 47.63956, -122.39300 47.6..."
3,53033006800,68,177.438771,2834.0,4,"POLYGON ((-122.35070 47.63994, -122.35071 47.6..."
4,53033006000,60,324.567432,5150.0,5,"POLYGON ((-122.34279 47.64320, -122.34267 47.6..."


In [60]:
test = pop_2010.NAME10.unique()

In [61]:
test_sort = np.sort(test.astype(float))
test_sort

array([  1.  ,   2.  ,   3.  ,   4.01,   4.02,   5.  ,   6.  ,   7.  ,
         8.  ,   9.  ,  10.  ,  11.  ,  12.  ,  13.  ,  14.  ,  15.  ,
        16.  ,  17.01,  17.02,  18.  ,  19.  ,  20.  ,  21.  ,  22.  ,
        24.  ,  25.  ,  26.  ,  27.  ,  28.  ,  29.  ,  30.  ,  31.  ,
        32.  ,  33.  ,  34.  ,  35.  ,  36.  ,  38.  ,  39.  ,  40.  ,
        41.  ,  42.  ,  43.01,  43.02,  44.  ,  45.  ,  46.  ,  47.  ,
        48.  ,  49.  ,  50.  ,  51.  ,  52.  ,  53.01,  53.02,  54.  ,
        56.  ,  57.  ,  58.01,  58.02,  59.  ,  60.  ,  61.  ,  62.  ,
        63.  ,  64.  ,  65.  ,  66.  ,  67.  ,  68.  ,  69.  ,  70.  ,
        71.  ,  72.  ,  73.  ,  74.01,  74.02,  75.  ,  76.  ,  77.  ,
        78.  ,  79.  ,  80.01,  80.02,  81.  ,  82.  ,  83.  ,  84.  ,
        85.  ,  86.  ,  87.  ,  88.  ,  89.  ,  90.  ,  91.  ,  92.  ,
        93.  ,  94.  ,  95.  ,  96.  ,  97.01,  97.02,  98.  ,  99.  ,
       100.01, 100.02, 101.  , 102.  , 103.  , 104.01, 104.02, 105.  ,
      

In [62]:
pop_tracts = pd.merge(census_bounds, pop_2010, on = 'NAME10')

In [63]:
pop_tracts.head()

,OBJECTID_x,TRACT,TRACTCE10,GEOID10_x,NAME10,NAMELSAD10,ACRES_TOTAL_x,WATER,SHAPE_Length,SHAPE_Area,geometry_x,GEOID10_y,ACRES_TOTAL_y,Total_Population,OBJECTID_y,geometry_y
0,1,2500,002500,53033002500,25,Census Tract 25,243.219083,0,16442.968402,1.059462e+07,"POLYGON ((-122.29602 47.69023, -122.29608 47.6...",53033002500,243.219083,2802.0,1,"POLYGON ((-122.29602 47.69023, -122.29604 47.6..."
1,2,2600,002600,53033002600,26,Census Tract 26,307.584420,0,15936.137058,1.339838e+07,"POLYGON ((-122.30817 47.69031, -122.30947 47.6...",53033002600,307.584420,4660.0,2,"POLYGON ((-122.30817 47.69031, -122.30793 47.6..."
2,3,5600,005600,53033005600,56,Census Tract 56,737.511727,0,30337.294354,3.212601e+07,"POLYGON ((-122.39300 47.63956, -122.39421 47.6...",53033005600,737.511727,6382.0,3,"POLYGON ((-122.39300 47.63956, -122.39300 47.6..."
3,4,6800,006800,53033006800,68,Census Tract 68,177.438771,0,11275.376490,7.729233e+06,"POLYGON ((-122.35070 47.63994, -122.35130 47.6...",53033006800,177.438771,2834.0,4,"POLYGON ((-122.35070 47.63994, -122.35071 47.6..."
4,5,6000,006000,53033006000,60,Census Tract 60,324.567432,0,19729.922875,1.413816e+07,"POLYGON ((-122.34279 47.64320, -122.34280 47.6...",53033006000,324.567432,5150.0,5,"POLYGON ((-122.34279 47.64320, -122.34267 47.6..."


In [64]:
pop_tracts = pop_tracts[['NAME10', 'geometry_x', 'Total_Population']]

In [65]:
pop_tracts.rename(columns={'geometry_x':'geometry'})

,NAME10,geometry,Total_Population
0,25,"POLYGON ((-122.29602 47.69023, -122.29608 47.6...",2802.0
1,26,"POLYGON ((-122.30817 47.69031, -122.30947 47.6...",4660.0
2,56,"POLYGON ((-122.39300 47.63956, -122.39421 47.6...",6382.0
3,68,"POLYGON ((-122.35070 47.63994, -122.35130 47.6...",2834.0
4,60,"POLYGON ((-122.34279 47.64320, -122.34280 47.6...",5150.0
...,...,...,...
130,260.01,"MULTIPOLYGON (((-122.24083 47.50223, -122.2408...",106.0
131,5,"POLYGON ((-122.36093 47.73414, -122.36093 47.7...",3165.0
132,109,"POLYGON ((-122.33813 47.55574, -122.33828 47.5...",1287.0
133,112,"POLYGON ((-122.33499 47.54107, -122.33500 47.5...",3873.0


In [66]:
total_pop = pop_tracts['Total_Population'].sum()
total_pop

608660.0

In [67]:
pop_tracts['Pop_fraction'] = pop_tracts['Total_Population']/total_pop
pop_tracts.head()

,NAME10,geometry_x,Total_Population,Pop_fraction
0,25,"POLYGON ((-122.29602 47.69023, -122.29608 47.6...",2802.0,0.004604
1,26,"POLYGON ((-122.30817 47.69031, -122.30947 47.6...",4660.0,0.007656
2,56,"POLYGON ((-122.39300 47.63956, -122.39421 47.6...",6382.0,0.010485
3,68,"POLYGON ((-122.35070 47.63994, -122.35130 47.6...",2834.0,0.004656
4,60,"POLYGON ((-122.34279 47.64320, -122.34280 47.6...",5150.0,0.008461


In [69]:
years = list(range(2007, 2019))
years

[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [70]:
populations = [585436, 591870, 598539, 608660, 622694, 635928, 653588, 670109, 687386, 709631, 728661, 742235]

In [71]:
pop_by_year = dict(zip(years, populations))
pop_by_year

{2007: 585436,
 2008: 591870,
 2009: 598539,
 2010: 608660,
 2011: 622694,
 2012: 635928,
 2013: 653588,
 2014: 670109,
 2015: 687386,
 2016: 709631,
 2017: 728661,
 2018: 742235}

In [72]:
pop_by_year.get(2015)

687386

In [73]:
def est_tract_pop(year, pop_tracts, pop_by_year):
    pop_frac = pop_tracts['Pop_fraction'].values
    year_pop = pop_by_year.get(year)
    pop_tracts_year = pop_tracts
    pop_tracts_year['Total_Population'] = pop_frac*year_pop
    return pop_tracts_year

In [74]:
pop_tracts_2007 = est_tract_pop(2007, pop_tracts, pop_by_year)
pop_tracts_2007.head()

,NAME10,geometry_x,Total_Population,Pop_fraction
0,25,"POLYGON ((-122.29602 47.69023, -122.29608 47.6...",2695.087031,0.004604
1,26,"POLYGON ((-122.30817 47.69031, -122.30947 47.6...",4482.193277,0.007656
2,56,"POLYGON ((-122.39300 47.63956, -122.39421 47.6...",6138.488733,0.010485
3,68,"POLYGON ((-122.35070 47.63994, -122.35130 47.6...",2725.866040,0.004656
4,60,"POLYGON ((-122.34279 47.64320, -122.34280 47.6...",4953.496862,0.008461


In [68]:
total_pop_2007 = pop_tracts_2007['Total_Population'].sum()
total_pop_2007

585436.0

## The Ultimate DataFrame -- coming soon

**Bike Lanes:**

In [102]:
df_lanes.head()

,0,1,2
0,2004,100.02,138.15138427468574
1,2004,101,138.31049553972244
2,2004,102,57.39931473030976
3,2004,103,156.09571324091084
4,2004,105,160.7547674761567
